<a href="https://colab.research.google.com/github/jonmessier/Sales-Predictions/blob/main/Project_1_Part_6_Standalone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Project 1 - Part 6(Core)

This week, you will finalize your sales prediction project. The goal of this is to help the retailer understand the properties of products and outlets that play crucial roles in predicting sales.

- [ ] Your first task is to build a linear regression model to predict sales.
 - [ ] Build a linear regression model.
 - [ ] Evaluate the performance of your model based on r^2.
 - [ ] Evaluate the performance of your model based on rmse.
- [ ] Your second task is to build a regression tree model to predict sales.
 - [ ] Build a simple regression tree model.
 - [ ] Compare the performance of your model based on r^2.
 - [ ] Compare the performance of your model based on rmse.
- [ ] You now have tried 2 different models on your data set. You need to determine which model to implement.
 - [ ] Overall, which model do you recommend?
 - [ ] Justify your recommendation.
- [ ] To finalize this project, complete a README in your GitHub repository including:
 - [ ] An overview of the project
 - [ ] 2 relevant insights from the data (supported with reporting quality visualizations)
 - [ ] Summary of the model and its evaluation metrics
 - [ ] Final recommendations 

Here is a template you can use for your readme if you would like. You can look at the raw readme file to copy it if you want.

Please note:
- Do not include detailed technical processes or code snippets in your README. If readers want to know more technical details they should be able to easily find your notebook to learn more.
- Make sure your GitHub repository is organized and professional. Remember, this should be used to showcase your data science skills and abilities.

Commit all of your work to GitHub and turn in a link to your GitHub repo with your final project.

#Custom Functions

In [70]:
#Define an inspection function to report for duplicates, and Nan values
#remove duplicates and output list of nan counts and total
def df_inspect(df):
  if df.duplicated().sum() >>0:
    print(f'The total number of duplicates are : {df.duplicated().sum()}\n')
    df.drop_duplicates(inplace=True)
    print('All duplicate entries have been removed.\n')
  print(f'There are no duplicate entries.\n')
  #Nan values
  print(f'The total number of NaN-values is:{df.isna().sum().sum()}')
  print(f'The NaN-values are found in the following features:')
  print(df.isna().sum())
  #shape
  print(f'\nThere are {df.shape[0]} rows, and {df.shape[1]} columns.')
  print(f'The rows represent {df.shape[0]} observations, and the columns represent {df.shape[1]-1} features and 1 target variable.\n')
  print(df.info())
  print(f'\nThe column names are:\n {df.columns}')

In [71]:
# Create a function to take the true and predicted values
# and print MAE, MSE, RMSE, and R2 metrics
def evaluation_model(model, model_name='', 
                     x_train='X_train', x_test='X_test'):
  # Train
  tr_rmse = round(np.sqrt(mean_squared_error(y_train, model.predict(x_train))),4)
  tr_r2 = round(r2_score(y_train, model.predict(x_train)),7)
  print(f'{model_name} Train Scores')
  print(f'RMSE: {tr_rmse:,.4f} \nR2: {tr_r2:.4f}\n')

  # Test
  te_rmse = round(np.sqrt(mean_squared_error(y_test, model.predict(x_test))),4)
  te_r2 = round(r2_score(y_test, model.predict(x_test)),7)

  # Display the metrics for the model
  print(f'{model_name} Test Scores')
  print(f'RMSE: {te_rmse:,.4f} \nR2: {te_r2:.4f}\n')
  report = {'Model':model_name,'Train_rmse': tr_rmse, 'Train_R2':tr_r2,
            'Test_rmse': te_rmse, 'Test_R2':te_r2}
  return report

#Data/Class Import

In [72]:
# Imports
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_selector, make_column_transformer


# Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_selector
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer
from sklearn.impute import SimpleImputer

# Models
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor

# Regression Metrics
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

# Set global scikit-learn configuration 
from sklearn import set_config
# Display estimators as a diagram
set_config(display='diagram') # 'text' or 'diagram'}

In [73]:
url = 'https://drive.google.com/uc?id=1syH81TVrbBsdymLT_jl2JIf6IjPXtSQw'
df = pd.read_csv(url)

##Data Overview

In [74]:
df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


##Data Inspection/Cleanup

In [75]:
#use custom inspection function to review data
df_inspect(df)

There are no duplicate entries.

The total number of NaN-values is:3873
The NaN-values are found in the following features:
Item_Identifier                 0
Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

There are 8523 rows, and 12 columns.
The rows represent 8523 observations, and the columns represent 11 features and 1 target variable.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-n

>- No Duplicate entries
- We have NaN values in `Item_Weight` and `Outlet_Size` features.  We will fill these with a Simple Imputer
- Target Variable is `Item_outlet_Sales`
- Feature `Dtypes` look appropriate.
- Column names appear without inconsitancies/errors  

###Inspect Numerical

In [76]:
df.describe(include="number")

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Outlet_Sales
count,7060.000000,8523.000000,8523.000000,8523.000000,8523.000000
mean,12.857645,0.066132,140.992782,1997.831867,2181.288914
std,4.643456,0.051598,62.275067,8.371760,1706.499616
min,4.555000,0.000000,31.290000,1985.000000,33.290000
25%,8.773750,0.026989,93.826500,1987.000000,834.247400
50%,12.600000,0.053931,143.012800,1999.000000,1794.331000
75%,16.850000,0.094585,185.643700,2004.000000,3101.296400
max,21.350000,0.328391,266.888400,2009.000000,13086.964800


> No unusual values noted

###Inspect Categorical

In [77]:
categoricals = df.select_dtypes(include='object')

for col in categoricals.columns:
  print(col)
  print(categoricals[col].value_counts(), '\n')

Item_Identifier
FDW13    10
FDG33    10
NCY18     9
FDD38     9
DRE49     9
         ..
FDY43     1
FDQ60     1
FDO33     1
DRF48     1
FDC23     1
Name: Item_Identifier, Length: 1559, dtype: int64 

Item_Fat_Content
Low Fat    5089
Regular    2889
LF          316
reg         117
low fat     112
Name: Item_Fat_Content, dtype: int64 

Item_Type
Fruits and Vegetables    1232
Snack Foods              1200
Household                 910
Frozen Foods              856
Dairy                     682
Canned                    649
Baking Goods              648
Health and Hygiene        520
Soft Drinks               445
Meat                      425
Breads                    251
Hard Drinks               214
Others                    169
Starchy Foods             148
Breakfast                 110
Seafood                    64
Name: Item_Type, dtype: int64 

Outlet_Identifier
OUT027    935
OUT013    932
OUT049    930
OUT046    930
OUT035    930
OUT045    929
OUT018    928
OUT017    926
OUT010    55

>- Inconsistancy noted in `Item_Fat_Content`
 - Replace inconsistent Item_Fat_Content values

In [78]:
df.replace(to_replace=['LF', 'low fat'], value='Low Fat', inplace=True)
df.replace('reg','Regular', inplace=True)
df['Item_Fat_Content'].value_counts()

Low Fat    5517
Regular    3006
Name: Item_Fat_Content, dtype: int64

>Cleaned up inconsistancies

###Missing Values
We review the missing data, but do not make changes at this time.  

In [79]:
# Display the count of missing values by column
print(df.isna().sum())

Item_Identifier                 0
Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64


> We have missing data in both numerical (`Item_Weight:float64`) and categorical (`Outlet_Size:object`) column types.  These will be replaced with SimpleImputer

#Train Test Split

In [80]:
# Define features (X) and target (y)
target = 'Item_Outlet_Sales'
X = df.drop(columns = target).copy()
y = df[target].copy()

In [81]:
# Split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [82]:
X.shape

(8523, 11)

In [83]:
y.shape

(8523,)

#Prepare Data

##Instantiate Imputers

In [85]:
#Use a 'mean-value' strategy for missing numeric data
mean_imputer = SimpleImputer(strategy='mean')

#Use a most_frequent strategy for missing ordinal data
freq_imputer = SimpleImputer(strategy='most_frequent')

#For missing values with nominal data, replace with 'UNK'
missing_imputer = SimpleImputer(strategy='constant', fill_value='UNK')

##Encoders
- We will use a One Hot Encoder for Categorical:Nominal and an Ordinal Encoder for categorical:Ordinal.  Numeric values will be scalled with StandardScaler.
- Our ordinal entries need to be encoded to capture the realtive values in a non-language based style.

- `Item_Outlet_Size` - `Small:0`, `Medium:1`, `High:2`

In [86]:
#instantiate the StandardScaler, OneHotEncoder, OrdinalEncoder and Imputers
scaler = StandardScaler()

#OneHot Encoder for categorical - nominal
ohe = OneHotEncoder(sparse=False, handle_unknown='ignore')

#Ordinal encoder for categorical - ordinal data
os_labels = ['Small', 'Medium', 'High']

#handle_unknown is 'error' by default.  That's a good place to start
#but it may cause problems in a production model.  
ordinal = OrdinalEncoder(categories = os_labels)

##Instantiate Pipelines
Our pipelines pair the data type with the proper imputer and encoders.


In [87]:
# Setup the pipelines.  We pair the imputer with the Encoder
#numerical pipeline - mean_imputer/scaler encoder
num_pipeline = make_pipeline(mean_imputer, scaler)

#ordinal values -most frequent/ordinal encoder
ord_pipeline = make_pipeline(freq_imputer, ordinal)

#nominal values - missing imputer/ohe
nom_pipeline = make_pipeline(missing_imputer, ohe)

##Create Tuples
Use a tuple to pair the correct pipline with the data columns

In [88]:
# Create column lists for objects and a number selector
ordinal_cols = ['Outlet_Size']
nominal_cols = ['Item_Identifier',
                'Item_Fat_Content',
                'Item_Type',
                'Outlet_Identifier',
                'Outlet_Location_Type',
                'Outlet_Type']

num_selector = make_column_selector(dtype_include='number')

In [89]:
# Setup the tuples to pair the processors with the column selectors
numeric_tuple = (num_pipeline, num_selector)
ordinal_tuple = (ord_pipeline, ordinal_cols)
nominal_tuple = (nom_pipeline, nominal_cols)

In [90]:
# Instantiate the make column transformer.  Drop all columns not inlcuded in our selected lists
preprocessor = make_column_transformer(ordinal_tuple, 
                                       numeric_tuple, 
                                       nominal_tuple, 
                                       remainder='drop')
preprocessor

ColumnTransformer(transformers=[('pipeline-1',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('ordinalencoder',
                                                  OrdinalEncoder(categories=['Small',
                                                                             'Medium',
                                                                             'High']))]),
                                 ['Outlet_Size']),
                                ('pipeline-2',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer()),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7ff292e99340>),
                                ('pipeline-3',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(fill_value='UNK',
                                                                strategy='constant')),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False))]),
                                 ['Item_Identifier', 'Item_Fat_Content',
                                  'Item_Type', 'Outlet_Identifier',
                                  'Outlet_Location_Type', 'Outlet_Type'])])

In [91]:
# Fit the column transformer on the X_train
preprocessor.fit(X_train);

ValueError: ignored

##Transform Training and Test Data

In [92]:
# Transform the X_train and the X_test
X_train_transformed = preprocessor.transform(X_train)
X_test_transformed = preprocessor.transform(X_test)
X_train_transformed.shape

AttributeError: ignored

In [ ]:
X_test_transformed.shape

In [ ]:
#Check that all values have been imputed
np.isnan(X_train_transformed).sum()

#Model Data

##Linear Regression Model

In [47]:
# Create an instance of the model
lin_reg = LinearRegression()

# Create a model pipeline
lin_reg_pipe = make_pipeline(preprocessor, lin_reg)
lin_reg_pipe


Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline-1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('ordinalencoder',
                                                                   OrdinalEncoder(categories=['Small',
                                                                                              'Medium',
                                                                                              'High']))]),
                                                  ['Outlet_Size']),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer()),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7ff293027a30>),
                                                 ('pipeline-3',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(fill_value='UNK',
                                                                                 strategy='constant')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse=False))]),
                                                  ['Item_Fat_Content',
                                                   'Item_Identifier',
                                                   'Item_Type',
                                                   'Outlet_Identifier',
                                                   'Outlet_Location_Type',
                                                   'Outlet_Type'])])),
                ('linearregression', LinearRegression())])

In [33]:
# Fit the model
lin_reg_pipe.fit(X_train, y_train)

ValueError: ignored

####Evaluate Model

In [ ]:
lr_report = evaluation_model(model=lin_reg_pipe, model_name='Linear Regression',
                 x_train=X_train, x_test=X_test)

##Decision Tree Model

In [95]:
#Create the model
dec_tree = DecisionTreeRegressor(random_state = 42)

# Create a model pipeline
dec_tree_pipe = make_pipeline(preprocessor, dec_tree)
dec_tree_pipe

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline-1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('ordinalencoder',
                                                                   OrdinalEncoder(categories=['Small',
                                                                                              'Medium',
                                                                                              'High']))]),
                                                  ['Outlet_Size']),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer()),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),...
                                                 ('pipeline-3',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(fill_value='UNK',
                                                                                 strategy='constant')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse=False))]),
                                                  ['Item_Identifier',
                                                   'Item_Fat_Content',
                                                   'Item_Type',
                                                   'Outlet_Identifier',
                                                   'Outlet_Location_Type',
                                                   'Outlet_Type'])])),
                ('decisiontreeregressor',
                 DecisionTreeRegressor(random_state=42))])

In [96]:
#Fit the model
dec_tree.fit(X_train, y_train)

ValueError: ignored

####Evaluate Model

In [ ]:
dt_report = evaluation_model(model=dec_tree_pipe, model_name='Decision Tree',
                 x_train=X_train, x_test=X_test)

##Compare Models

In [ ]:
report=[lr_report, dt_report]
reportdf = pd.DataFrame(report)
reportdf

#Recomendation